# 1ère NSI - Python avancé #1

## Références et Valeurs


### Cas des types simples : tout va bien !

In [93]:
a = 5
b = a
print("a=",a,"\tb=",b)

a= 5 	b= 5


**a et b ont la même valeur**.

In [94]:
b=7
print("a =",a,"\tb =",b)

a = 5 	b = 7


Si on **change la valeur de b, a reste inchangé**. Évidemment me direz-vous !

### Cas des Listes : ⚠️ Danger ! ⚠️

Réalisons le même code avec des **listes**

In [95]:
liste1 = [1,2,3]
liste2 = liste1
print("liste1 =",liste1,"\tliste2 =",liste2)

liste1 = [1, 2, 3] 	liste2 = [1, 2, 3]


**liste1 et liste2 ont la même valeur**. Mais **modifions liste2** ???

In [96]:
liste2 += [4]
print("liste1 =",liste1,"\tliste2 =",liste2)

liste1 = [1, 2, 3, 4] 	liste2 = [1, 2, 3, 4]


⚠️ **Si on modifie liste2, liste1 est modifiée aussi !** ⚠️ Que c'est-il passé ???

Pour comprendre, écrivons une fonction qui retourne ***l'adresse*** ou la **référence** de la variable en mémoire :

In [97]:
def adr(l: list):
    return hex(id(l))

Regardons les adresses de liste1 et liste2 :

In [98]:
print("Adresse de liste1:",adr(liste1))
print("Adresse de liste2:",adr(liste2))

Adresse de liste1: 0x7f59143ebcd0
Adresse de liste2: 0x7f59143ebcd0


Les deux variables **liste1 et liste2 pointent vers la même adresse en mémoire** : leur contenu est donc toujours **identique**. En effet, si je modifie l'une, je modifie l'autre puisqu'elles **pointent sur le même contenu en mémoire**.

Python **ne recopie pas le contenu de liste1 dans liste2**, il se contente de **faire pointer la variable liste2 sur l'emplacement mémoire de liste1**.

Si on souhaite **recopier** liste1 dans liste2 et que les 2 variables **restent indépendantes** avec des **adresses différentes**, il faut indiquer **explicitement la copie** come ceci :

In [99]:
liste1 = [1,2,3]
liste2 = liste1.copy()
print("liste1 =",liste1,"\tliste2 =",liste2)
print("Adresse de liste1:",adr(liste1))
print("Adresse de liste2:",adr(liste2))

liste1 = [1, 2, 3] 	liste2 = [1, 2, 3]
Adresse de liste1: 0x7f59144816e0
Adresse de liste2: 0x7f591440ec80


In [100]:
liste2 += [4]
print("liste1 =",liste1,"\tliste2 =",liste2)

liste1 = [1, 2, 3] 	liste2 = [1, 2, 3, 4]


Cette fois les deux variables sont bien **distinctes** et occupent **2 places en mémoire différentes**. Elles sont donc **indépendantes**.

##  Variables Immuables (immutable) et Muables (mutable)

### Immuables

Les variables de type **int, float, bool, str** sont **immuables** c'est-à-dire **pas modifiables** !

C'est évidemment **étrange** car vous avez bien modifié le contenu de certaines variables de type int dans vos programmes...

Une variable **immuable ne peut pas changer le contenu de son emplacement en mémoire**. Donc, quand on souhaite modifier une variable **immuable**, par exemple si on écrit `a+=2`, Python **crée une nouvelle variable avec le même nom ailleurs en mémoire**.

Vérifions cela :

In [101]:
a = 5
print("Valeur de a:", a, "\t\tAdresse de la variable a: ", adr(a))
a+=2
print("Valeur de a:", a, "\t\tAdresse de la variable a: ", adr(a))

Valeur de a: 5 		Adresse de la variable a:  0x7f5919d03460
Valeur de a: 7 		Adresse de la variable a:  0x7f5919d034a0


La variable a est bien **modifiée, mais ce n'est pas la même**, c'est une **nouvelle** variable a !

À chaque fois que Python **modifie la valeur d'une variable immuable, il en crée une nouvelle copie ! C'est inefficace** mais apporte **plus de sécurité** car cela évite de modifier des variables sans le vouloir.

### Muables

Les variables de type **list, set, dict** sont **muables** c'est-à-dire sont **modifiables sans changer de référence en mémoire**.

Une variable **muable peut changer de contenu en gardant la même référence en mémoire**.

Vérifions cela aussi :

In [102]:
liste = [1,2,3]
print("Valeur de liste:", liste, "\t\tAdresse de la variable liste: ", adr(liste))
liste += [2]
print("Valeur de liste:", liste, "\t\tAdresse de la variable liste: ", adr(liste))

Valeur de liste: [1, 2, 3] 		Adresse de la variable liste:  0x7f591447d820
Valeur de liste: [1, 2, 3, 2] 		Adresse de la variable liste:  0x7f591447d820


La variable liste est **muable** elle est donc bien **modifiée** directement, **en gardant la même référence en mémoire** !

Les variables **muables** engendrent parfois des **bugs car elles peuvent être modifiées par erreur de programmation**. Par contre cette gestion est **plus efficace** car on ne **recopie pas la variable à chaque modification**.

## Passage de paramètres aux fonctions

Il existe le même problème quand on passe des arguments aux fonctions :

- Pour les variables de **type int, float, bool, str**, il n'y a **pas d'effet de bord** car ces types sont **immuables**. La **variable passée en paramètre est recopiée ailleurs en mémoire dès quelle est modifiée**.

- Pour les variables de **type list, set, dict**, il y'a des **effets de bord** car ces types sont **muables**. La **fonction** reçoit **l'adresse de la variable passée en argument** et **peut** donc en **modifier le contenu** car la variable est **muable**.

**Effet de bord** : quand une fonction modifie le contenu d'une variable qui appartient au contexte appelant.

Illustrons cela avec 2 exemples. Nous créons une fonction **test** qui va ajouter 2 à la variable passée en argument.

In [103]:
def test(variable):
    print("Adresse de la variable passée en argument: ", adr(variable))
    
    if type(variable) is int:
        variable += 4
    elif type(variable) is list:
        variable.append(4)
    else :
        print("fonction de test à utiliser avec des int ou des list")
        
    print("Adresse de la variable une fois modifiée: ", adr(variable))

### Passage d'une variable immuable : Pas d'effet de bord !

In [104]:
a = 5
print("Adresse de la variable a: ", adr(a))
test(a)
print("a=",a)

Adresse de la variable a:  0x7f5919d03460
Adresse de la variable passée en argument:  0x7f5919d03460
Adresse de la variable une fois modifiée:  0x7f5919d034e0
a= 5


⚠️ La fonction test reçoit **l'adresse de la variable a** mais quand elle veut **modifier** la variable, il s'en **crée une copie à une autre adresse** : la variable **a n'est donc pas modifiée** ! **Ici, pas d'effet de bord !**

### Passage d'une variable muable : ⚠️ Effet de bord ! ⚠️

In [105]:
liste = [1,2,3]
print("Adresse de la variable liste: ", adr(liste))
test(liste)
print("liste=",liste)

Adresse de la variable liste:  0x7f591440b5f0
Adresse de la variable passée en argument:  0x7f591440b5f0
Adresse de la variable une fois modifiée:  0x7f591440b5f0
liste= [1, 2, 3, 4]


⚠️ La fonction test reçoit **l'adresse de la variable liste** et peut **modifier** la variable car elle est **muable** ! **Ici il y'a effet de bord !** ⚠️